In [1]:
from salishsea_tools import viz_tools
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import cmocean.cm as cm
import cv2

In [2]:
physics_202111 = xr.open_dataset('/ocean/cdonaldson/compare_daily_river/SalishSeaCast_day_avg_physics_20190101_20191231.nc')
year = 2019

In [14]:
phys_arr = physics_202111.vosaline.to_numpy()

# open the mesh mask and use it to mask the land
mesh = xr.open_dataset('../../../../MEOPAR/grid/mesh_mask202108.nc')
# the mask is 1 where there is water, we want the opposite.  The meshmask has an extra dimension, hence the [0]
tmask = 1 - mesh.tmask[0]
tmask = tmask[0:20, :, :]

In [11]:
phys_1d = physics_202111.vosaline[0,:,:,:]
phys_1d.shape

(20, 898, 398)

In [15]:
tmask.shape

(20, 898, 398)

## Birgit's method

In [25]:
# mask array of grid box thickness based on where there is land (tmask):
e3t_mask = np.ma.masked_where((tmask[:,:,:] < 0.1), phys_1d[:,:,:])

# find index of grid cell just above the ocean floor by finding the edges for the array along the z-axis:
ind = np.array(np.ma.notmasked_edges(e3t_mask[:,:,:], axis=0)) # e3t shape: (z,x,y)

# create array of shape (isize, jsize) containing bottom grid cell indices:
# index_bottom = ind[1][0][:].reshape(isize,jsize)

In [26]:
ind[1][0][:].shape

(286312,)

## It looks like I can maybe downscale the array first?

Tereza's paper describes a method in which stations are selected throughout the SOG, which seems like a pixel-selection method of decreasing resolution. I think I want to do averages to downscale? But I should be careful since I ultimately only want the near-coastline points in the end.

In [25]:
# ## from ChatGPT, turn this into my own function?

# # Create a sample array (replace this with your own data)
# original_array = phys_arr[0,0,:,:]

# # Define the scale factor (e.g., 2 for halving the dimensions)
# scale_factor = 10

# # Calculate the new dimensions for the downsampled array
# new_height = original_array.shape[0] // scale_factor
# new_width = original_array.shape[1] // scale_factor

# # Create an empty array for the downsampled result
# downsampled_array = np.zeros((new_height, new_width), dtype=original_array.dtype)

# # Downscale the array by taking the average of each group of pixels
# for y in range(new_height):
#     for x in range(new_width):
#         y_start = y * scale_factor
#         y_end = y_start + scale_factor
#         x_start = x * scale_factor
#         x_end = x_start + scale_factor
#         downsampled_array[y, x] = np.mean(original_array[y_start:y_end, x_start:x_end])

In [5]:
# # Load your matrix as an image (assuming you have NaN values for land)
# data_matrix = phys_arr

# # Create a binary mask where land (NaN values) are set to 0 and water is set to 255
# binary_mask = np.where(np.isnan(data_matrix), 0, 255).astype(np.uint8)

# # Find contours in the binary mask
# contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# # Assuming you want to extract the outermost contour as the coastline
# coastline_contour = max(contours, key=cv2.contourArea)

# # Extract coordinates from the coastline contour
# coastline_coordinates = coastline_contour.reshape(-1, 2)

# # Now, coastline_coordinates contains the coordinates of the coastline.

: 

: 